# PandasDataFrameOutputParser

데이터 조작 및 분석을 위한 강력한 도구, 구조화된 데이터를 효과적으로 다루기 위한 포괄적은 도구세트이며, 이를통해 데이터 정제, 변환 빛 분석이 가능하다

출력 파서는 사용자가 임의의 DataFreame을 지정하여, 해당 DataFrame에서 데이터를 추출하고, 이를 dictionary형태로 조회할 수 있게 한다.

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import pprint
from typing import Any, Dict

import pandas as pd
from langchain.output_parsers import PandasDataFrameOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [9]:
#model = ChatOpenAI(model="gpt-4o-mini") 4o는 안되더라. 왠진 모르겠다.
model = ChatOpenAI(model="gpt-3.5-turbo")

`format_parser_output` 함수는 파서 출력을 사전 형식으로 변환하고, 출력 형식을 지정하는데 사용.

In [10]:
# 출력 목적으로만 사용
def format_parser_output(parser_output: Dict[str, Any]) -> None:
    for key in parser_output.keys():
        # 각 키의 값을 딕셔너리로 변환
        parser_output[key] = parser_output[key].to_dict()
    #이쁘게 출력
    return pprint.PrettyPrinter(width=4, compact=True).pprint(parser_output)

In [11]:
# Pandas DataFreame 로드 및 정의.
df = pd.read_csv("../../data/titanic.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [12]:
parser = PandasDataFrameOutputParser(dataframe=df)

print(parser.get_format_instructions())
# Here are the possible columns은, Data freame을 가지고 columns들을 추가를 해주었다.

The output should be formatted as a string as the operation, followed by a colon, followed by the column or row to be queried on, followed by optional array parameters.
1. The column names are limited to the possible columns below.
2. Arrays must either be a comma-separated list of numbers formatted as [1,3,5], or it must be in range of numbers formatted as [0..4].
3. Remember that arrays are optional and not necessarily required.
4. If the column is not in the possible columns or the operation is not a valid Pandas DataFrame operation, return why it is invalid as a sentence starting with either "Invalid column" or "Invalid operation".

As an example, for the formats:
1. String "column:num_legs" is a well-formatted instance which gets the column num_legs, where num_legs is a possible column.
2. String "row:1" is a well-formatted instance which gets row 1.
3. String "column:num_legs[1,2]" is a well-formatted instance which gets the column num_legs for rows 1 and 2, where num_legs is a p

In [13]:
df_query = "Age Column을 조회해 주세요"

prompt = PromptTemplate(
    template = 
    """
Answer the user query. 
{format_instructions}
{query}
""",
    input_variables=["query"], # 입력변수 설정
    partial_variables={ # 부분변수 설정
        "format_instructions": parser.get_format_instructions()
    }
)

chain = prompt | model | parser

# 출력
parser_output = chain.invoke({"query": df_query}) #pandas query
format_parser_output(parser_output)

{'Age': {0: 22.0,
         1: 38.0,
         2: 26.0,
         3: 35.0,
         4: 35.0,
         5: nan,
         6: 54.0,
         7: 2.0,
         8: 27.0,
         9: 14.0,
         10: 4.0,
         11: 58.0,
         12: 20.0,
         13: 39.0,
         14: 14.0,
         15: 55.0,
         16: 2.0,
         17: nan,
         18: 31.0,
         19: nan}}


In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
print(llm.invoke("RAG를 할때 시간이 오래걸리는 경우는"))

content='RAG (Retrieval-Augmented Generation) 모델을 사용할 때 시간이 오래 걸리는 경우는 여러 가지 이유가 있을 수 있습니다. 다음은 일반적인 원인들입니다:\n\n1. **데이터베이스 크기**: 사용하고 있는 데이터베이스가 매우 크면, 적절한 정보를 검색하는 데 시간이 더 걸릴 수 있습니다. 데이터베이스의 크기가 클수록 검색 알고리즘이 더 많은 데이터를 처리해야 하므로 시간이 소요됩니다.\n\n2. **검색 알고리즘**: RAG 모델에서 정보를 검색하기 위해 사용하는 알고리즘의 효율성에 따라 시간이 달라질 수 있습니다. 비효율적인 검색 방법이나 복잡한 쿼리는 결과를 얻는 데 더 많은 시간을 필요로 할 수 있습니다.\n\n3. **모델 크기 및 복잡성**: RAG 모델 자체가 크고 복잡할수록, 특히 파라미터가 많고 레이어가 깊은 경우, 생성 과정에서 시간이 더 걸릴 수 있습니다.\n\n4. **하드웨어 성능**: 모델이 실행되는 서버나 클라우드의 하드웨어 성능도 중요한 요소입니다. CPU나 GPU의 성능이 낮으면 처리 시간이 길어질 수 있습니다.\n\n5. **배치 처리**: 여러 개의 요청을 동시에 처리하는 배치 처리 방식이 아닌 경우, 각 요청을 순차적으로 처리하느라 시간이 더 걸릴 수 있습니다.\n\n6. **네트워크 지연**: 클라우드 기반의 시스템을 사용하는 경우, 네트워크 지연이나 속도 저하가 모델의 응답 시간을 늦출 수 있습니다.\n\n7. **사전 처리 및 후처리**: 입력 데이터를 준비하거나 출력 결과를 후처리하는 과정에서도 시간이 소요될 수 있습니다.\n\n이러한 요인들을 고려하여 성능을 최적화하거나, 필요에 따라 하드웨어를 업그레이드하거나 알고리즘을 개선하는 등의 방법을 통해 처리 시간을 단축할 수 있습니다.' response_metadata={'token_usage': {'completion_tokens': 418, 'prompt_tokens': 18, 'total_tokens': 436}, 'model

In [ ]:
print(llm.invoke("RAG를 할때 시간이 오래걸리는 경우는 임베딩모델, VectorDB, LLM모델을을 들 수 있을까?"))

content='네, RAG (Retrieval-Augmented Generation) 시스템에서 시간이 오래 걸리는 경우는 여러 요소에 의해 영향을 받을 수 있습니다. 그 중에서 임베딩 모델, VectorDB, LLM (대형 언어 모델) 모두 중요한 역할을 합니다.\n\n1. **임베딩 모델**: 입력 데이터를 벡터로 변환하는 과정에서 시간이 많이 소요될 수 있습니다. 특히 대량의 데이터를 처리할 경우, 임베딩 모델의 속도가 전체 시스템의 성능에 큰 영향을 미칠 수 있습니다.\n\n2. **VectorDB**: 벡터 데이터베이스에서 유사한 벡터를 검색하는 과정도 시간이 걸릴 수 있습니다. 데이터베이스의 크기와 사용되는 인덱싱 기술, 검색 알고리즘의 효율성에 따라 검색 시간이 달라질 수 있습니다. 대규모 데이터셋에서는 특히나 검색 시간이 증가할 수 있습니다.\n\n3. **LLM 모델**: 대형 언어 모델을 사용하는 경우, 모델의 크기와 복잡성에 따라 추론 시간이 길어질 수 있습니다. LLM은 대량의 파라미터를 포함하고 있기 때문에, 입력에 대한 응답을 생성하는 데 시간이 걸릴 수 있습니다. 또한, 모델을 배포하는 환경(예: GPU, TPU 등)에 따라 성능이 달라질 수 있습니다.\n\n이 외에도 데이터 전처리, 시스템의 하드웨어 성능, 병렬 처리의 효율성 등 여러 요인이 RAG 시스템의 전체 수행 시간에 영향을 줄 수 있습니다. 따라서 각 구성 요소의 성능을 최적화하는 것이 중요합니다.' response_metadata={'token_usage': {'completion_tokens': 340, 'prompt_tokens': 38, 'total_tokens': 378}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_9b0abffe81', 'finish_reason': 'stop', 'logprobs': None} id='run-ae1d90c6-a9cd-432e-a98b-5f6bb9b77